In [15]:
import os
import string
import random

import cv2
import numpy as np
from luxai_s2.env import LuxAI_S2
import matplotlib.pyplot as plt
from rich import print
from agent import Agent

In [16]:
def animate(imgs, _return=True):
    video_name = ''.join(random.choice(string.ascii_letters) for i in range(18))+'.webm'
    height, width, layers = imgs[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'VP90')
    video = cv2.VideoWriter(video_name, fourcc, 10, (width,height))

    for img in imgs:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        video.write(img)
    video.release()
    if _return:
        from IPython.display import Video
        return Video(video_name)
    

def interact(env, agents, steps):
    # reset our env
    obs = env.reset(seed=41)
    np.random.seed(0)
    imgs = []
    step = 0
    # Note that as the environment has two phases, we also keep track a value called 
    # `real_env_steps` in the environment state. The first phase ends once `real_env_steps` is 0 and used below

    # iterate until phase 1 ends
    while env.state.real_env_steps < 0:
        if step >= steps:
            break
        actions = {}
        for player in env.agents:
            o = obs[player]
            if step == 0:
                a = agents[player].bid_policy(step, o)
            else:
                a = agents[player].factory_placement_policy(step, o)
            actions[player] = a
        step += 1
        obs, rewards, dones, infos = env.step(actions)
        imgs += [env.render("rgb_array", width=640, height=640)]

    done = False
    while not done:
        if step >= steps:
            break
        actions = {}
        for player in env.agents:
            o = obs[player]
            a = agents[player].act(step, o)
            actions[player] = a
        step += 1
        obs, rewards, dones, infos = env.step(actions)
        imgs += [env.render("rgb_array", width=640, height=640)]
        done = dones["player_0"] and dones["player_1"]
    return animate(imgs)

In [4]:
env = LuxAI_S2() # create the environment object
obs = env.reset(seed=41) # resets an environment with a seed

In [13]:
agents = {player: Agent(player, env.state.env_cfg) for player in env.agents}

In [19]:
interact(env, agents, 200)

OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'


In [22]:
!tar --exclude *.ipynb* --exclude __pycache__ --exclude *.gz -vczf submission.tar.gz *

a agent.py
a logs
a logs/exp_1
a logs/exp_1/models
a logs/exp_1/models/best_model.zip
a lux
a lux/kit.py
a lux/config.py
a lux/cargo.py
a lux/__init__.py
a lux/factory.py
a lux/team.py
a lux/utils.py
a lux/unit.py
a main.py
a requirements.txt
a train.py
a wrappers
a wrappers/controllers.py
a wrappers/obs_wrappers.py
a wrappers/__init__.py


In [23]:
!luxai-s2 main.py main.py -v 2 -s 101 -o replay.html

-10: player_1 tried to perform an action in the early phase when it is not its turn right now.
51: Invalid movement action for unit [0] unit_2 UnitType.HEAVY at (7, 43) - Tried to move to (7, 42) requiring 85 power
52: Invalid Dig Action for unit [0] unit_2 UnitType.HEAVY at (7, 43) - Tried to dig requiring 60 power
53: Invalid Dig Action for unit [0] unit_2 UnitType.HEAVY at (7, 43) - Tried to dig requiring 60 power
54: Invalid movement action for unit [0] unit_2 UnitType.HEAVY at (7, 43) - Tried to move to (7, 42) requiring 85 power
55: Invalid Dig Action for unit [0] unit_2 UnitType.HEAVY at (7, 43) - Tried to dig requiring 60 power
56: Invalid Dig Action for unit [0] unit_2 UnitType.HEAVY at (7, 43) - Tried to dig requiring 60 power
57: Invalid Dig Action for unit [0] unit_2 UnitType.HEAVY at (7, 43) - Tried to dig requiring 60 power
59: Invalid Dig Action for unit [0] unit_2 UnitType.HEAVY at (7, 43) - Tried to dig requiring 60 power
60: Invalid Dig Action for unit [0] unit_2 Unit

In [1]:
import IPython
IPython.display.HTML(filename='replay.html')